# 네이버 영화 리뷰 KOR

In [1]:
import pandas as pd
!pip install transformers
from transformers import AutoTokenizer, ElectraForSequenceClassification, AdamW
import numpy as np

from nltk.tokenize import sent_tokenize
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import nltk
nltk.download('punkt')

import urllib
from nltk.corpus import stopwords  
import random

#nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')

     |████████████████████████████████| 1.5MB 7.2MB/s 
     |████████████████████████████████| 890kB 48.1MB/s 
     |████████████████████████████████| 2.9MB 38.5MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=ffc171cdbb52824eeacaada11082d0b8b1ed11f3436d6bb2ea77fca7eb79d1ef
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [2]:
!pip install konlpy

     |████████████████████████████████| 19.4MB 57.7MB/s 
     |████████████████████████████████| 92kB 10.7MB/s 
     |████████████████████████████████| 460kB 44.1MB/s 
  Found existing installation: tweepy 3.6.0
    Uninstalling tweepy-3.6.0:
      Successfully uninstalled tweepy-3.6.0
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [3]:
from konlpy.tag import Okt
from konlpy.tag import Kkma
from konlpy.tag import Hannanum
from konlpy.tag import Komoran
from konlpy.tag import Twitter

kkma = Kkma()
okt = Okt()
komoran = Komoran()
hannanum = Hannanum()
twitter = Twitter()

/usr/local/lib/python3.6/dist-packages/konlpy/tag/_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


데이터 읽기 - 참고 : https://colab.research.google.com/drive/1JZ-pXlmgRIYHm8yPLYY68Q28l9OYAL6H?usp=sharing

In [4]:
!wget https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt
!wget https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt

--2020-12-24 14:34:05--  https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4893335 (4.7M) [text/plain]
Saving to: ‘ratings_test.txt’

ratings_test.txt    100%[===================>]   4.67M  28.7MB/s    in 0.2s    

2020-12-24 14:34:05 (28.7 MB/s) - ‘ratings_test.txt’ saved [4893335/4893335]

--2020-12-24 14:34:05--  https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14628807 (14M) [text/plain]
Saving to: ‘ratings

In [5]:
t_dataset = pd.read_csv("ratings_train.txt", sep='\t').dropna(axis=0) 
test_dataset = pd.read_csv("ratings_test.txt", sep='\t').dropna(axis=0) 

코드 참고 : https://colab.research.google.com/drive/18nAZ_uqxP5pb8hKBHUc-UuTapAzdcqgE#scrollTo=**f8hRnRUG_ss8**

In [10]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(t_dataset['document'])

In [11]:
print(X_train_counts.shape)
print(X_train_counts[0])
print(count_vect.vocabulary_)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [12]:
from sklearn.feature_extraction.text import TfidfTransformer
tf_transformer = TfidfTransformer(use_idf=True).fit(X_train_counts)
X_train_tf = tf_transformer.transform(X_train_counts)

In [14]:
print(X_train_tf.shape)
print(X_train_tf[0])

(149995, 293366)
  (0, 248358)	0.6303226931788337
  (0, 246232)	0.26458741210117614
  (0, 99567)	0.5193460471477966
  (0, 71119)	0.5128025810517561


In [15]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train_tf, t_dataset['label'])

In [16]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train_tf, t_dataset['label'])

In [18]:
from xgboost import XGBClassifier
from sklearn.pipeline import Pipeline
pipe_clf = Pipeline([
    ('vect', CountVectorizer(ngram_range=(1, 2), tokenizer = komoran.morphs)), #tokenizer = nltk.word_tokenize)),#(stop_words=stopwords.words('english'))),
    #('tfidf', TfidfTransformer()),
    ('clf', XGBClassifier(colsample_bylevel=0.8, olsample_bytree=0.8,max_depth=8,learning_rate=0.2, n_estimators=500)),
])
pipe_clf.fit(t_dataset['document'], t_dataset['label'])

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 2), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<bound method Komoran...
                               colsample_bylevel=0.8, colsample_bynode=1,
                               colsample_bytree=1, gamma=0, learning_rate=0.2,
                               max_delta_step=0, max_depth=8,
                               min_child_weight=1, missing=None,
                               n_estimators=500, n_jobs

In [19]:
import numpy as np

predicted = pipe_clf.predict(test_dataset['document'])
result = np.mean(predicted == test_dataset['label'])
print(result)

0.842410544632678


In [20]:
from collections import Counter
print(Counter(t_dataset['label']))
print(Counter(test_dataset['label']))

Counter({0: 75170, 1: 74825})
Counter({1: 25171, 0: 24826})


In [21]:
from sklearn import metrics

result = metrics.classification_report(
    y_true = test_dataset['label'],
    y_pred = predicted)#,
    #target_names = test_f['emotion'])
print(result)

              precision    recall  f1-score   support

           0       0.82      0.87      0.85     24826
           1       0.86      0.82      0.84     25171

    accuracy                           0.84     49997
   macro avg       0.84      0.84      0.84     49997
weighted avg       0.84      0.84      0.84     49997



In [22]:
k_test = pd.read_csv('ko_data.csv', encoding= 'cp949')
k_test.head()

FileNotFoundError: ignored

In [ ]:
predicted = pipe_clf.predict(k_test['Sentence'])
print(predicted)

In [ ]:
pd.DataFrame(predicted).to_csv("naver_2020511028.csv")